In [2]:
    import io, IPython, functools, importlib_resources
    shell = __import__('IPython').get_ipython()

In [ ]:
    def alt(req, resp): return __import__('altair').Chart(**resp.json())

    def sqlite(req, resp):
        parsed = urllib.parse.urlparse(response.url)
        # do something for remotes here
        scheme, sep, path = resp.url.partition('://')
        connection = __import__('ibis').sqlite.connect(path)
        return F"{response.url} has tables named: {', '.join(connection.list_tables())}"
    
    def read_pandas(attr, request, response): 
        return getattr(__import__('pandas'), F"read_{attr}")(io.BytesIO(response.content)).sample(4)


    def iframe(req): return IPython.display.IFrame(req.url, '100%', '600')

In [8]:
    def load_ipython_extension(shell):
        annotation = shell.annotation_formatter            
        annotation.for_type_by_ext(
            'py', lambda req, resp: IPython.display.Code(
                resp.content, language='python'
            )
        )
        annotation.for_type_by_name(None, None, iframe)
        annotation.for_type_by_ext('com', iframe)
        annotation.for_type_by_ext(
            'py', lambda req, resp: IPython.display.Code(
                resp.text, language='python'
            )
        )
        annotation.for_type_by_ext('md', lambda req, resp: IPython.display.Markdown(resp.text))
        for key in "csv feather parquet".split():
            annotation.for_type_by_ext(key, functools.partial(read_pandas, key))
        annotation.for_type_by_ext('json', lambda req, resp: IPython.display.JSON(resp.json()))
        annotation.for_type_by_ext('ipynb', lambda req, resp: IPython.display.JSON(resp.json()))
        annotation.for_type_by_ext('yaml', lambda req, resp: IPython.display.JSON(Yaml%resp.text))
        annotation.for_type_by_ext('geojson', lambda req, resp: IPython.display.GeoJSON(resp.json()))

        annotation.for_type_by_ext('html', lambda req, resp: IPython.display.HTML(F"""<iframe srcdoc="{__import__('html').escape(resp.text)}" width="100%" height="600"/>"""))
        for key in "jpg png bmp".split():
            annotation.for_type_by_ext('jpg', lambda req, resp: IPython.display.Image(data=resp.content))
            annotation.for_type_by_ext('png', lambda req, resp: IPython.display.Image(data=resp.content))    
        annotation.for_type_by_ext('vl', alt)    
        annotation.for_type_by_ext('sqlite', sqlite)    
        annotation.for_type_by_name('', None, lambda request: IPython.display.Markdown(F"* {request.url}"))        

In [8]:
    def unload_ipython_extension(shell): ...
        